<a href="https://colab.research.google.com/github/saatweek/Coronavirus_tweets_NLP_Text_Classification/blob/master/Corona_Tweets_Classification_(BERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mounting Google Drive to this colab notebook

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing all the dependencies

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
!pip install ktrain
import ktrain
from ktrain import text
ktrain.__version__

     |████████████████████████████████| 25.3MB 130kB/s 
     |████████████████████████████████| 983kB 51.0MB/s 
     |████████████████████████████████| 266kB 42.3MB/s 
     |████████████████████████████████| 1.3MB 53.2MB/s 
     |████████████████████████████████| 471kB 52.5MB/s 
     |████████████████████████████████| 2.9MB 26.9MB/s 
     |████████████████████████████████| 1.1MB 53.5MB/s 
     |████████████████████████████████| 890kB 54.6MB/s 
  Created wheel for ktrain: filename=ktrain-0.24.1-cp36-none-any.whl size=25274041 sha256=d65d0c432a7674bbd22d94e118af3cabfd4a9606c1a300d082caeae140088eb3
  Stored in directory: /root/.cache/pip/wheels/46/93/f6/f82e69b760f7320ad0a030366f5b140ba2be4e7740cc0b0b31
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=a98d260b704008f04595bdff6137089324a3a9b9426342fe08f88a73fa109398
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

'0.24.1'

In [ ]:
#training file and test file paths

train_csv_path = "/content/drive/My Drive/Colab Notebooks/Personal Projects/Corona tweets/Corona_NLP_train.csv"
test_csv_path = '/content/drive/My Drive/Colab Notebooks/Personal Projects/Corona tweets/Corona_NLP_test.csv'

In [ ]:
#reading the training_file through pandas

training_dataframe = pd.read_csv(train_csv_path, encoding = "ISO-8859-1")
training_dataframe.dropna()
testing_dataframe = pd.read_csv(test_csv_path)
testing_dataframe.dropna()

#printing the training data
print(training_dataframe.shape)
training_dataframe.head()

(41157, 6)


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
#printing the testing data

print(testing_dataframe.shape)
testing_dataframe.head()

(3798, 6)


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
#making empty lists for storing traning and testing data

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

In [ ]:
#selecting the training sentences and appending each data to the previously created training_sentences list
for sentences in training_dataframe.iloc[:, -2]:
    training_sentences.append(sentences)

for labels in training_dataframe.iloc[:, -1]:
    labels = labels.replace(' ', '')
    training_labels.append(labels)

for sentences in testing_dataframe.iloc[:, -2]:
    testing_sentences.append(sentences)

for labels in testing_dataframe.iloc[:, -1]:
    labels = labels.replace(' ', '')
    testing_labels.append(labels)

In [ ]:
categories = ['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive']

trn, val, preproc = text.texts_from_array(x_train=training_sentences, y_train=training_labels,
                                          x_test=testing_sentences, y_test=testing_labels,
                                          class_names=categories,
                                          preprocess_mode='distilbert',
                                          maxlen=350)




preprocessing train...
language: en
train sequence lengths:
	mean : 31
	95percentile : 48
	99percentile : 52


/usr/local/lib/python3.6/dist-packages/ktrain/utils.py:588: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 33
	95percentile : 50
	99percentile : 54


task: text classification


In [ ]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [ ]:
model = text.text_classifier('distilbert', train_data=trn, preproc=preproc)

Is Multi-Label? False
maxlen is 350



done.


In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [ ]:
history = learner.fit_onecycle(1e-5, 5)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/5
6860/6860 [==============================] - 1758s 256ms/step - loss: 1.1024 - accuracy: 0.5393 - val_loss: 0.7839 - val_accuracy: 0.6922
Epoch 2/5
6860/6860 [==============================] - 1765s 257ms/step - loss: 0.6286 - accuracy: 0.7616 - val_loss: 0.5698 - val_accuracy: 0.7838
Epoch 3/5
6860/6860 [==============================] - 1766s 257ms/step - loss: 0.4317 - accuracy: 0.8435 - val_loss: 0.4653 - val_accuracy: 0.8291
Epoch 4/5
6860/6860 [==============================] - 1762s 257ms/step - loss: 0.2847 - accuracy: 0.9008 - val_loss: 0.4750 - val_accuracy: 0.8415
Epoch 5/5
6860/6860 [==============================] - 1760s 257ms/step - loss: 0.1872 - accuracy: 0.9373 - val_loss: 0.5065 - val_accuracy: 0.8418


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
x = np.linspace(1,100, 100)
y1 = history.history['accuracy']
y2 = history.history['val_accuracy']
fig.add_trace(go.Scatter(x = x, y = y1, name = 'accuracy')),
fig.add_trace(go.Scatter(x = x, y = y2, name = 'val_accuracy', ))
fig.update_layout(xaxis_title = 'Epochs', yaxis_title = 'accuracy', title = 'Accuracy of Model')
fig.show()